Аналитика следующая:
* Слабые места (куда уходит больше всего)
    - Топ категорий по расходам
    - Доли категорий
    - Всплески расходов по дням (rule-based + ML anomaly)
* Регулярные платежи
    - Группировка по категориям
    - Проверка на периодичность (weekly, monthly)
    - Итог: список "похоже на подписку/регулярный платёж"
* Экономия и "сколько можно отложить"
    - Политика экономии (правила и модель)
    - Расчёт `potential_savings_total`
    - Разложение экономии по категориям и кластерам
    - Итог: "если снизить X%, то можно отложить Y рублей за период N"

In [2]:
from __future__ import annotations

import re
import math
from typing import Dict, Optional, List

import numpy as np
import pandas as pd

from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sympy import pprint


In [3]:
FILE_PATH = "assets/03 июля 2025 - 03 января 2026 (2).xlsx"

In [4]:
def _find_header_row(df_raw: pd.DataFrame, required_cols: List[str], max_scan: int = 200) -> int:
    """
    В выписках часто есть служебные строки до таблицы.
    Ищем строку, где встречаются названия колонок (Дата/Сумма/Категория/Описание).
    """
    best_i, best_score = 0, -1
    for i in range(min(len(df_raw), max_scan)):
        row = df_raw.iloc[i].astype(str).str.strip().tolist()
        row_set = set(row)

        score = sum(1 for c in required_cols if c in row_set)
        if score > best_score:
            best_score = score
            best_i = i

        # если нашли почти всё — это оно
        if score >= max(2, len(required_cols) - 1):
            return i

    if best_score <= 0:
        raise ValueError("Не удалось найти строку заголовков в файле.")
    return best_i


def load_bank_xlsx(path: str) -> pd.DataFrame:
    df_raw = pd.read_excel(path, header=None)

    required = ["Дата", "Сумма", "Категория", "Описание"]
    header_row = _find_header_row(df_raw, required_cols=required)

    df = pd.read_excel(path, header=header_row)
    df = df.dropna(how="all").reset_index(drop=True)

    # убираем мусорные Unnamed колонки
    df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]
    return df


df_raw = load_bank_xlsx(FILE_PATH)
f_raw.head(10)

,Дата операции,Дата проводки,Код,Категория,Описание,Сумма в валюте счета,Статус
0,03.07.2025,03.07.2025,C170307252005946,Прочие операции,Категория: Перевод по СБП.Перевод C17030725200...,1 548,Выполнен
1,04.07.2025,04.07.2025,C460407250088613,Прочие операции,Категория: Перевод по СБП.Перевод C46040725008...,-614,Выполнен
2,04.07.2025,04.07.2025,C460407250086843,Прочие операции,Категория: Перевод по СБП.Перевод C46040725008...,-1 616,Выполнен
3,04.07.2025,04.07.2025,C460407250033028,Прочие операции,Категория: Перевод по СБП.Перевод C46040725003...,-841,Выполнен
4,04.07.2025,04.07.2025,OP1ED2574002545N,Прочие операции,"Перечисление средств в рамках услуги ""Копилка ...",-1,NaN
5,05.07.2025,05.07.2025,C460507250072255,Прочие операции,Категория: Перевод по СБП.Перевод C46050725007...,-200,Выполнен
6,05.07.2025,05.07.2025,C170507250215936,Прочие операции,Категория: Перевод по СБП.Перевод C17050725021...,5 000,Выполнен
7,06.07.2025,06.07.2025,C460607250077032,Прочие операции,Категория: Перевод по СБП.Перевод C46060725007...,-688,Выполнен
8,06.07.2025,06.07.2025,OP1ED2576001GG6P,Прочие операции,"Перечисление средств в рамках услуги ""Копилка ...","-51,22",NaN
9,06.07.2025,06.07.2025,OP1ED2576001G2FO,Прочие операции,"Перечисление средств в рамках услуги ""Копилка ...","-88,10",NaN


In [5]:
def _to_datetime_safe(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce", dayfirst=True)


def _parse_money_series(s: pd.Series) -> pd.Series:
    """
    Приводим суммы к float:
    "1 234,56" -> 1234.56
    """
    if s.dtype.kind in "if":
        return s.astype(float)

    x = s.astype(str)
    x = x.str.replace("\u202f", " ", regex=False).str.replace("\xa0", " ", regex=False)
    x = x.str.replace(" ", "", regex=False)
    x = x.str.replace(",", ".", regex=False)
    x = x.str.replace(r"[^0-9\.\-]", "", regex=True)

    def _safe_float(v: str) -> float:
        try:
            if v in ("", "-", ".", "-."):
                return np.nan
            return float(v)
        except Exception:
            return np.nan

    return x.apply(_safe_float)


def _pick_col(columns: List[str], *variants: str) -> Optional[str]:
    low = {c: str(c).strip().lower() for c in columns}

    # точное совпадение
    for v in variants:
        v = v.lower()
        for c, lc in low.items():
            if lc == v:
                return c

    # частичное
    for v in variants:
        v = v.lower()
        for c, lc in low.items():
            if v in lc:
                return c

    return None


def normalize_transactions(df: pd.DataFrame) -> pd.DataFrame:
    c_date = _pick_col(df.columns.tolist(), "дата", "дата операции", "дата транзакции")
    c_amount = _pick_col(df.columns.tolist(), "сумма", "сумма операции", "amount")
    c_cat = _pick_col(df.columns.tolist(), "категория", "category")
    c_desc = _pick_col(df.columns.tolist(), "описание", "описание операции", "назначение", "контрагент", "merchant", "details")

    if c_date is None or c_amount is None:
        raise ValueError(f"Не нашёл обязательные колонки даты/суммы. Колонки: {list(df.columns)}")

    out = pd.DataFrame()
    out["op_date"] = _to_datetime_safe(df[c_date])
    out["amount"] = _parse_money_series(df[c_amount])

    out["category"] = df[c_cat].astype(str).str.strip() if c_cat else "Без категории"
    out["description"] = df[c_desc].astype(str).str.strip() if c_desc else ""

    # чистка
    out = out.dropna(subset=["op_date", "amount"]).reset_index(drop=True)

    # направление (по умолчанию: расходы отрицательные)
    out["direction"] = np.where(out["amount"] < 0, "expense", "income")
    out["abs_amount"] = out["amount"].abs()

    out["month"] = out["op_date"].dt.to_period("M").astype(str)

    # merchant — грубая нормализация строки для группировок
    out["merchant"] = (
        out["description"]
        .fillna("")
        .str.lower()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(r"[^a-zа-я0-9 &\-\.\_]", " ", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    return out


tx = normalize_transactions(df_raw)
tx.head()


,op_date,amount,category,description,direction,abs_amount,month,merchant
0,2025-07-03,1548.0,Прочие операции,Категория: Перевод по СБП.Перевод C17030725200...,income,1548.0,2025-07,категория перевод по сбп.перевод c170307252005...
1,2025-07-04,-614.0,Прочие операции,Категория: Перевод по СБП.Перевод C46040725008...,expense,614.0,2025-07,категория перевод по сбп.перевод c460407250088...
2,2025-07-04,-1616.0,Прочие операции,Категория: Перевод по СБП.Перевод C46040725008...,expense,1616.0,2025-07,категория перевод по сбп.перевод c460407250086...
3,2025-07-04,-841.0,Прочие операции,Категория: Перевод по СБП.Перевод C46040725003...,expense,841.0,2025-07,категория перевод по сбп.перевод c460407250033...
4,2025-07-04,-1.0,Прочие операции,"Перечисление средств в рамках услуги ""Копилка ...",expense,1.0,2025-07,перечисление средств в рамках услуги копилка д...


In [6]:
# Ячейка 6 — data quality checks
tx.isna().mean().sort_values(ascending=False), tx.shape, tx["direction"].value_counts()

(op_date        0.0
 amount         0.0
 category       0.0
 description    0.0
 direction      0.0
 abs_amount     0.0
 month          0.0
 merchant       0.0
 dtype: float64,
 (1451, 8),
 direction
 expense    1132
 income      319
 Name: count, dtype: int64)

In [7]:
# Ячейка 7 — summary

def compute_summary(tx: pd.DataFrame) -> Dict:
    if tx.empty:
        return {"period": None, "income": 0.0, "expense": 0.0, "net": 0.0, "transactions": 0}

    period = (tx["op_date"].min().date().isoformat(), tx["op_date"].max().date().isoformat())

    income = float(tx.loc[tx["direction"] == "income", "amount"].sum())
    expense = float(tx.loc[tx["direction"] == "expense", "amount"].sum())  # обычно отрицательное
    net = income + expense

    return {
        "period": period,
        "income": round(income, 2),
        "expense": round(expense, 2),
        "net": round(net, 2),
        "transactions": int(len(tx)),
    }

summary = compute_summary(tx)
summary


{'period': ('2025-07-03', '2026-01-03'),
 'income': 3400138.7,
 'expense': -3405597.75,
 'net': -5459.05,
 'transactions': 1451}

In [8]:
# Ячейка 8 — top categories

def top_categories(tx: pd.DataFrame, n: int = 10) -> pd.DataFrame:
    exp = tx[tx["direction"] == "expense"].copy()
    if exp.empty:
        return pd.DataFrame(columns=["category", "expense_abs", "share"])

    by = exp.groupby("category", dropna=False)["abs_amount"].sum().sort_values(ascending=False)
    total = float(by.sum()) if float(by.sum()) > 0 else 1.0

    out = by.head(n).reset_index()
    out.columns = ["category", "expense_abs"]
    out["share"] = out["expense_abs"] / total
    return out

topcats = top_categories(tx, n=10)
topcats


,category,expense_abs,share
0,Прочие операции,2967277.45,0.871294
1,Электроника,114768.00,0.033700
2,Товары для дома,72520.90,0.021295
3,Кафе и рестораны,59894.20,0.017587
4,Финансовые операции,44231.85,0.012988
5,Одежда и обувь,38656.00,0.011351
6,Образование,22287.00,0.006544
7,Животные,19081.90,0.005603
8,"Телефон, интернет, ТВ",15453.00,0.004538
9,Путешествия,13158.00,0.003864


In [9]:
# Ячейка 9 — recurring detection

def detect_recurring(
        tx: pd.DataFrame,
        min_occurrences: int = 3,
        tolerance_days: int = 5,
        amount_tolerance: float = 0.15,
) -> pd.DataFrame:
    exp = tx[tx["direction"] == "expense"].copy()
    if exp.empty:
        return pd.DataFrame()

    rows = []
    for (m, c), g in exp.groupby(["merchant", "category"]):
        if len(g) < min_occurrences:
            continue

        g = g.sort_values("op_date")
        deltas = np.diff(g["op_date"].values).astype("timedelta64[D]").astype(int)
        if len(deltas) < (min_occurrences - 1):
            continue

        med_delta = int(np.median(deltas))
        weekly_like = abs(med_delta - 7) <= tolerance_days
        monthly_like = abs(med_delta - 30) <= tolerance_days
        if not (weekly_like or monthly_like):
            continue

        amounts = g["abs_amount"].values.astype(float)
        med_amount = float(np.median(amounts))
        if med_amount <= 0:
            continue

        ok_amount = np.all(np.abs(amounts - med_amount) <= (amount_tolerance * med_amount))
        if not ok_amount:
            continue

        rows.append({
            "merchant": m,
            "category": c,
            "occurrences": int(len(g)),
            "median_amount": round(med_amount, 2),
            "period_days_median": med_delta,
            "period_type": "weekly" if weekly_like else "monthly",
            "last_date": str(g["op_date"].max().date()),
        })

    out = pd.DataFrame(rows)
    if out.empty:
        return out
    return out.sort_values(["period_type", "median_amount"], ascending=[True, False]).reset_index(drop=True)

recurring = detect_recurring(tx)
recurring.head(20)


,merchant,category,occurrences,median_amount,period_days_median,period_type,last_date
0,внутрибанковский перевод между счетами исупов ...,Прочие операции,5,100.0,7,weekly,2025-08-05


In [10]:
# Ячейка 10 — spikes rule + ML

def daily_expenses(tx: pd.DataFrame) -> pd.DataFrame:
    exp = tx[tx["direction"] == "expense"].copy()
    if exp.empty:
        return pd.DataFrame(columns=["day", "expense_abs"])

    d = exp.groupby(exp["op_date"].dt.date)["abs_amount"].sum().reset_index()
    d.columns = ["day", "expense_abs"]
    return d

def detect_spikes_rule(daily: pd.DataFrame, z: float = 2.5) -> pd.DataFrame:
    if daily.empty:
        return pd.DataFrame()

    mu = float(daily["expense_abs"].mean())
    sd = float(daily["expense_abs"].std(ddof=0) or 0.0)

    out = daily.copy()
    out["z"] = 0.0 if sd == 0 else (out["expense_abs"] - mu) / sd
    return out[out["z"] >= z].sort_values("z", ascending=False).reset_index(drop=True)

def detect_spikes_ml(daily: pd.DataFrame, contamination: float = 0.03, random_state: int = 42) -> pd.DataFrame:
    if daily.empty or len(daily) < 15:
        return pd.DataFrame()

    X = daily[["expense_abs"]].values.astype(float)
    model = IsolationForest(n_estimators=300, contamination=contamination, random_state=random_state)
    model.fit(X)

    out = daily.copy()
    out["score"] = model.decision_function(X)  # ниже => более аномально
    out["anomaly"] = model.predict(X)          # -1 => аномалия
    return out[out["anomaly"] == -1].sort_values("score").reset_index(drop=True)

daily = daily_expenses(tx)
spikes_rule = detect_spikes_rule(daily, z=2.5)
spikes_ml = detect_spikes_ml(daily, contamination=0.03)

daily.head(), spikes_rule.head(10), spikes_ml.head(10)


(          day  expense_abs
 0  2025-07-03      1068.85
 1  2025-07-04      3757.73
 2  2025-07-05       200.00
 3  2025-07-06      4602.00
 4  2025-07-07      6306.00,
           day  expense_abs         z
 0  2025-12-25    123543.00  3.992251
 1  2025-10-31    123024.00  3.972524
 2  2025-11-20    121173.18  3.902176
 3  2025-11-21    106099.94  3.329257
 4  2025-12-04    105511.98  3.306910
 5  2025-10-29     94182.70  2.876295
 6  2025-12-29     92064.99  2.795803
 7  2025-09-01     90426.88  2.733540
 8  2025-12-17     87980.26  2.640546,
           day  expense_abs     score  anomaly
 0  2025-11-20    121173.18 -0.098334       -1
 1  2025-12-25    123543.00 -0.083354       -1
 2  2025-10-31    123024.00 -0.067293       -1
 3  2025-11-21    106099.94 -0.038370       -1
 4  2025-12-04    105511.98 -0.037376       -1
 5  2025-12-20     75512.00 -0.001401       -1)

In [11]:
# Ячейка 11 — clustering

def cluster_expenses(tx: pd.DataFrame, n_clusters: int = 8, min_docs: int = 50, random_state: int = 42) -> pd.DataFrame:
    exp = tx[tx["direction"] == "expense"].copy()
    if exp.empty or len(exp) < min_docs:
        return pd.DataFrame()

    texts = (exp["merchant"].fillna("") + " " + exp["category"].fillna("")).str.strip()
    texts = texts.replace("", np.nan).dropna()

    exp = exp.loc[texts.index].copy()
    texts = texts.loc[exp.index]

    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=2)
    X = vectorizer.fit_transform(texts)

    k = min(n_clusters, max(2, int(math.sqrt(len(exp)))))
    model = KMeans(n_clusters=k, random_state=random_state, n_init="auto")
    labels = model.fit_predict(X)

    exp["cluster"] = labels

    # агрегат по кластерам
    agg = exp.groupby("cluster")["abs_amount"].sum().sort_values(ascending=False).reset_index()
    agg.columns = ["cluster", "expense_abs"]

    # 3 ключевых термина для объяснимости
    feature_names = np.array(vectorizer.get_feature_names_out())
    centers = model.cluster_centers_

    keywords = []
    for cl in agg["cluster"].values:
        top_idx = np.argsort(centers[cl])[-5:][::-1]
        keywords.append(", ".join(feature_names[top_idx][:3]))

    agg["keywords"] = keywords
    return agg

clusters = cluster_expenses(tx, n_clusters=8, min_docs=50)
clusters.head(15)


,cluster,expense_abs,keywords
0,0,1317859.78,"перевод, на 79127458900, 79127458900"
1,3,1148110.51,"сч, со сч, исупов со"
2,1,526927.68,"ru, 2025, 2025 место"
3,4,176290.60,"mcc 3990, 3990, yandex"
4,6,92857.00,"на 79859437117, 79859437117 без, 79859437117"
5,7,75873.75,"moscow dostavka, dostavka, 5411 прочие"
6,5,63887.20,"рестораны, кафе рестораны, кафе"
7,2,3791.23,"перечисление средств, перечисление, со счета"


In [12]:
# Ячейка 12 — savings estimation (simple policy)

DEFAULT_SAVING_RULES = [
    (r"кафе|ресторан|фастфуд|еда вне дома", 0.20, "часть покупок заменить домашней едой"),
    (r"такси", 0.15, "часть поездок заменить общественным транспортом"),
    (r"подписк", 0.30, "проверить подписки и отменить лишние"),
    (r"маркет|доставка|онлайн", 0.10, "сократить импульсные заказы"),
    (r"развлеч|кино|игр", 0.10, "планировать развлечения заранее"),
]

def estimate_savings(tx: pd.DataFrame, rules=DEFAULT_SAVING_RULES) -> pd.DataFrame:
    exp = tx[tx["direction"] == "expense"].copy()
    if exp.empty:
        return pd.DataFrame()

    cat_sum = exp.groupby("category")["abs_amount"].sum().reset_index()
    cat_sum.columns = ["category", "expense_abs"]

    rows = []
    for _, r in cat_sum.iterrows():
        cat = str(r["category"])
        val = float(r["expense_abs"])

        matched = None
        for pat, reduction, comment in rules:
            if re.search(pat, cat.lower()):
                matched = (reduction, comment)
                break

        if matched is None:
            continue

        reduction, comment = matched
        rows.append({
            "category": cat,
            "expense_abs": round(val, 2),
            "target_reduction": reduction,
            "potential_saving": round(val * reduction, 2),
            "comment": comment
        })

    out = pd.DataFrame(rows)
    if out.empty:
        return out
    return out.sort_values("potential_saving", ascending=False).reset_index(drop=True)

savings = estimate_savings(tx)
savings.head(20), float(savings["potential_saving"].sum()) if not savings.empty else 0.0


(           category  expense_abs  target_reduction  potential_saving  \
 0  Кафе и рестораны      59894.2               0.2          11978.84   
 
                                 comment  
 0  часть покупок заменить домашней едой  ,
 11978.84)

In [13]:
# Ячейка 13 — report builder

def build_report(tx: pd.DataFrame) -> Dict:
    summary = compute_summary(tx)
    topcats = top_categories(tx, n=10)
    recurring = detect_recurring(tx)
    daily = daily_expenses(tx)
    spikes_rule = detect_spikes_rule(daily, z=2.5)
    spikes_ml = detect_spikes_ml(daily, contamination=0.03)
    clusters = cluster_expenses(tx, n_clusters=8, min_docs=50)
    savings = estimate_savings(tx)

    potential_total = float(savings["potential_saving"].sum()) if isinstance(savings, pd.DataFrame) and not savings.empty else 0.0

    return {
        "summary": summary,
        "top_categories": topcats,
        "recurring": recurring,
        "daily_expenses": daily,
        "spikes_rule": spikes_rule,
        "spikes_ml": spikes_ml,
        "clusters": clusters,
        "savings": savings,
        "potential_savings_total": round(potential_total, 2),
    }

report = build_report(tx)
report["summary"]


{'period': ('2025-07-03', '2026-01-03'),
 'income': 3400138.7,
 'expense': -3405597.75,
 'net': -5459.05,
 'transactions': 1451}

In [16]:
# Ячейка 14 — telegram text rendering

def render_telegram_text(report: Dict, top_n: int = 6) -> str:
    s = report["summary"]
    period = s["period"]
    period_txt = f"{period[0]} — {period[1]}" if period else "неизвестен"

    lines = []
    lines.append(f"📊 Анализ расходов за период: {period_txt}")
    lines.append(f"• Операций: {s['transactions']}")
    lines.append(f"• Доходы: {s['income']:.2f}")
    lines.append(f"• Расходы: {s['expense']:.2f}")
    lines.append(f"• Сальдо: {s['net']:.2f}")
    lines.append("")

    topcats = report["top_categories"]
    if isinstance(topcats, pd.DataFrame) and not topcats.empty:
        lines.append("🔝 Топ категорий расходов:")
        for _, r in topcats.head(top_n).iterrows():
            lines.append(f"• {r['category']}: {r['expense_abs']:.2f} (доля {r['share']*100:.1f}%)")
        lines.append("")

    recurring = report["recurring"]
    if isinstance(recurring, pd.DataFrame) and not recurring.empty:
        lines.append("🔁 Похоже на регулярные платежи (топ-5):")
        for _, r in recurring.head(5).iterrows():
            pt = "еженедельно" if r["period_type"] == "weekly" else "ежемесячно"
            lines.append(f"• {r['merchant']} — {r['median_amount']:.2f} ({pt}), раз: {r['occurrences']}")
        lines.append("")

    savings = report["savings"]
    if isinstance(savings, pd.DataFrame) and not savings.empty:
        lines.append("💡 Где можно сэкономить (оценка):")
        for _, r in savings.head(5).iterrows():
            lines.append(f"• {r['category']}: ~{r['potential_saving']:.2f} → {r['comment']}")
        lines.append("")
        lines.append(f"💰 Потенциально можно отложить: ~{report['potential_savings_total']:.2f} за весь период")
        lines.append("")

    # всплески
    spikes_rule = report["spikes_rule"]
    spikes_ml = report["spikes_ml"]

    if isinstance(spikes_rule, pd.DataFrame) and not spikes_rule.empty:
        lines.append("⚠️ Дни с аномально высокими расходами (правило):")
        for _, r in spikes_rule.head(3).iterrows():
            lines.append(f"• {r['day']}: {r['expense_abs']:.2f}")
        lines.append("")

    if isinstance(spikes_ml, pd.DataFrame) and not spikes_ml.empty:
        lines.append("🤖 Дни-аномалии (ML):")
        for _, r in spikes_ml.head(3).iterrows():
            lines.append(f"• {r['day']}: {r['expense_abs']:.2f}")
        lines.append("")

    return "\n".join(lines).strip()

telegram_text = render_telegram_text(report)
print(telegram_text)


📊 Анализ расходов за период: 2025-07-03 — 2026-01-03
• Операций: 1451
• Доходы: 3400138.70
• Расходы: -3405597.75
• Сальдо: -5459.05

🔝 Топ категорий расходов:
• Прочие операции: 2967277.45 (доля 87.1%)
• Электроника: 114768.00 (доля 3.4%)
• Товары для дома: 72520.90 (доля 2.1%)
• Кафе и рестораны: 59894.20 (доля 1.8%)
• Финансовые операции: 44231.85 (доля 1.3%)
• Одежда и обувь: 38656.00 (доля 1.1%)

🔁 Похоже на регулярные платежи (топ-5):
• внутрибанковский перевод между счетами исупов г. с..со сч та 40817810205601502126 на сч т 30601810800139496794 — 100.00 (еженедельно), раз: 5

💡 Где можно сэкономить (оценка):
• Кафе и рестораны: ~11978.84 → часть покупок заменить домашней едой

💰 Потенциально можно отложить: ~11978.84 за весь период

⚠️ Дни с аномально высокими расходами (правило):
• 2025-12-25: 123543.00
• 2025-10-31: 123024.00
• 2025-11-20: 121173.18

🤖 Дни-аномалии (ML):
• 2025-11-20: 121173.18
• 2025-12-25: 123543.00
• 2025-10-31: 123024.00
